## Importing Libraries


In [1]:
import pandas as pd

## Data Pre-Processing

In [2]:
df = pd.read_csv('train.csv')
val = pd.read_csv('test.csv')

In [3]:
print(len(df))
print(len(val))

104322
2002


In [4]:
import re

# clean text from noise
def clean_text(text):
    # filter to allow only alphabets
    text = re.sub(r'[^a-zA-Z\']', ' ', text)
    
    # remove Unicode characters
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    
    # convert to lowercase to maintain consistency
    text = text.lower()
       
    return text

df['text'] = df.text.apply(clean_text)
val['text'] = val.text.apply(clean_text)

In [5]:
import fasttext
DATA_FILE = './fasttext_data.txt'
#  Storing Dataset in txt
with open(DATA_FILE, 'w', encoding="utf-8") as fp:
    for i in range(len(df)):
        fp.write('__label__%s %s\n' % (df['label'][i],
                                       df['text'][i]))

with open('./prediction.results', 'w', encoding="utf-8") as fp:
    for i in range(len(val)):
        fp.write('__label__%s\n' % val['label'][i])
with open('./prediction.test', 'w', encoding="utf-8") as fp:
    for i in range(len(val)):
        fp.write('%s\n' % (val['text'][i]))

## Training and Testing fastText model

In [6]:
import fasttext
model = fasttext.train_supervised(input='fasttext_data.txt', epoch=30, lr=0.4, wordNgrams=5, loss='ova')

In [14]:
model.save_model("model_bullying.bin")

In [8]:
classifier = fasttext.load_model("model_bullying.bin")

In [9]:
with open('prediction.test', 'r') as f:
    x_test = [line.strip() for line in f]

In [10]:
y_pred = []
for i in x_test:
    y_pred.append(classifier.predict(i)[0][0])

In [11]:
with open('prediction.results', 'r') as f:
    y_true = [line.strip() for line in f]

In [12]:
from sklearn.metrics import accuracy_score, confusion_matrix
cm = confusion_matrix(y_true, y_pred)
accuracy_score(y_true, y_pred)

0.9410589410589411

In [13]:

cm

array([[961,  40],
       [ 78, 923]], dtype=int64)